In [ ]:
import numpy as np
import pandas as pd

## 读取数据

In [ ]:
df_hs300 = pd.read_csv("01_data/hs300.csv")
df_hs300["交易日期"] = pd.to_datetime(df_hs300["交易日期"])
df_hs300_c_s = pd.read_csv("01_data/hs300_constituent_stocks.csv")
df_hs300_c_s["交易日期"] = pd.to_datetime(df_hs300_c_s["交易日期"])
df_hs300_c_s_p = pd.read_csv("01_data/hs300_constituent_stocks_predict.csv")
df_hs300_c_s_p = df_hs300_c_s_p.rename(columns={'date': '交易日期','code': '股票代码'})
df_hs300_c_s_p["交易日期"] = pd.to_datetime(df_hs300_c_s_p["交易日期"])

## 合并真实值与预测

In [ ]:
df_hs300_c_s_p_merge = pd.merge(df_hs300_c_s, df_hs300_c_s_p, on = ["交易日期", "股票代码"], how = "inner")

## 策略与评估

### 1 全仓购买预测值排序为1的股票

In [ ]:
# 先按股票代码排序，方便获取预测值第一的股票下一天的收益
df_hs300_c_s_p_merge_new = df_hs300_c_s_p_merge.sort_values(by = ['股票代码','交易日期'],ascending=True)

In [ ]:
# 获取预测值第一的股票下一天的收益
df_hs300_c_s_p1_rate = df_hs300_c_s_p_merge_new[df_hs300_c_s_p_merge_new['pred'].shift(1) == 1.0].sort_values(by = ['交易日期'],ascending=True)[['交易日期','涨跌幅（%）']]
df_hs300_c_s_p1_rate['pred1收益率'] = df_hs300_c_s_p1_rate['涨跌幅（%）'] + 1

In [ ]:
# 获取沪深300涨跌幅
df_hs300_rate = df_hs300[pd.to_datetime('2021-01-04') <= df_hs300["交易日期"]][['交易日期','涨跌幅(%)']]
df_hs300_rate['hs300收益率'] = df_hs300_rate['涨跌幅(%)']/100 + 1

In [ ]:
# 计算选取
df_hs300_c_s_p1_rate = pd.merge(df_hs300_c_s_p1_rate, df_hs300_rate, on = ["交易日期"], how = "inner")
df_hs300_c_s_p1_rate['超额收益'] = df_hs300_c_s_p1_rate['pred1收益率'] - df_hs300_c_s_p1_rate['hs300收益率']